In [59]:
import numpy as np
import pycuda.autoinit
import pycuda.driver as cuda
from pycuda.compiler import SourceModule
from pycuda import gpuarray
import time

In [60]:
cuda_code = """
#include <curand.h>
#include <curand_kernel.h>

extern "C" {
    __global__ void fitness_kernel(float *population, float *points_x, float *points_y, 
                                 float *fitnesses, int num_individuals, int num_points, int degree) {
        int idx = blockIdx.x * blockDim.x + threadIdx.x;
        if (idx >= num_individuals) return;

        float error = 0.0f;
        for (int j = 0; j < num_points; j++) {
            float x = points_x[j];
            float y = points_y[j];
            float approximation = 0.0f;
            
            for (int k = 0; k <= degree; k++) {
                approximation += population[idx * (degree + 1) + k] * powf(x, k);
            }
            
            error += powf(y - approximation, 2);
        }
        fitnesses[idx] = error;
    }

    __global__ void mutation_kernel(float *population, float *new_population,
                                  int num_individuals, int num_genes,
                                  float mutation_rate, unsigned int seed) {
        int idx = blockIdx.x * blockDim.x + threadIdx.x;
        if (idx >= num_individuals * num_genes) return;

        // Calculate indices
        int individual_idx = idx / num_genes;
        int gene_idx = idx % num_genes;
        
        // Initialize random state
        curandState_t state;
        curand_init(seed + idx, individual_idx, gene_idx, &state);
        
        // Copy gene
        float gene = population[idx];
        
        // Apply mutation with probability mutation_rate
        if (curand_uniform(&state) < mutation_rate) {
            gene += curand_normal(&state) * 0.1f;
        }
        
        new_population[idx] = gene;
    }
}
"""

In [61]:
class CUDAGeneticAlgorithm:
    def __init__(self, population_size, num_genes, points_x, points_y, degree):
        self.population_size = population_size
        self.num_genes = num_genes
        self.points_x = points_x
        self.points_y = points_y
        self.degree = degree
        
        # Compile CUDA kernels
        try:
            self.mod = SourceModule(cuda_code, no_extern_c=True)
            self.fitness_kernel = self.mod.get_function("fitness_kernel")
            self.mutation_kernel = self.mod.get_function("mutation_kernel")
            print("CUDA kernels compiled successfully")
        except Exception as e:
            print(f"Error compiling CUDA kernels: {e}")
            raise
        
        # Initialize population on GPU
        self.population = gpuarray.to_gpu(
            np.random.normal(0, 1, (population_size, num_genes)).astype(np.float32)
        )
        
        # Transfer data points to GPU
        self.d_points_x = gpuarray.to_gpu(points_x.astype(np.float32))
        self.d_points_y = gpuarray.to_gpu(points_y.astype(np.float32))
        
        # Allocate memory for fitness values
        self.fitnesses = gpuarray.zeros(population_size, dtype=np.float32)
        
        # Calculate grid and block dimensions
        self.block_size = 256
        self.grid_size = (population_size + self.block_size - 1) // self.block_size
        
    def calculate_fitness(self):
        self.fitness_kernel(
            self.population, self.d_points_x, self.d_points_y,
            self.fitnesses, np.int32(self.population_size),
            np.int32(len(self.points_x)), np.int32(self.degree),
            block=(self.block_size, 1, 1), grid=(self.grid_size, 1)
        )
        return self.fitnesses.get()
    
    def mutate(self, mutation_rate=0.1):
        new_population = gpuarray.zeros_like(self.population)
        
        mutation_block_size = 256
        mutation_grid_size = (self.population_size * self.num_genes + mutation_block_size - 1) // mutation_block_size
        
        self.mutation_kernel(
            self.population, new_population,
            np.int32(self.population_size), np.int32(self.num_genes),
            np.float32(mutation_rate), np.uint32(np.random.randint(1000000)),
            block=(mutation_block_size, 1, 1), grid=(mutation_grid_size, 1)
        )
        
        self.population = new_population
    
    def evolve(self, generations=100, mutation_rate=0.1):
        for generation in range(generations):
            # Calculate fitness
            fitnesses = self.calculate_fitness()
            
            # Print best fitness
            if generation % 10 == 0:
                print(f"Generation {generation}: Best fitness = {np.min(fitnesses):.6f}")
            
            # Apply mutation
            self.mutate(mutation_rate)
        
        # Return best individual
        fitnesses = self.calculate_fitness()
        best_idx = np.argmin(fitnesses)
        return self.population.get()[best_idx], fitnesses[best_idx]

In [62]:
# Generate sample data points (x^2 function with some noise)
x = np.linspace(-5, 5, 100)
y = x**2 + np.random.normal(0, 0.5, x.shape)
    
    # Initialize and run genetic algorithm
ga = CUDAGeneticAlgorithm(
    population_size=1000,
    num_genes=3,  # Degree 2 polynomial coefficients
    points_x=x,
    points_y=y,
    degree=2
)
    
best_solution, best_fitness = ga.evolve(generations=100)
print("\nBest solution found:", best_solution)
print("Final fitness:", best_fitness)

CUDA kernels compiled successfully
Generation 0: Best fitness = 19.766119
Generation 10: Best fitness = 20.573874
Generation 20: Best fitness = 31.744631
Generation 30: Best fitness = 42.455147
Generation 40: Best fitness = 41.838524
Generation 50: Best fitness = 110.018127
Generation 60: Best fitness = 103.001396
Generation 70: Best fitness = 83.513512
Generation 80: Best fitness = 37.837734
Generation 90: Best fitness = 48.038269

Best solution found: [-0.5356288   0.05709771  1.0654552 ]
Final fitness: 48.148727
